In [6]:
import os
# Imports
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from scikeras.wrappers import KerasRegressor
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import torch
import torch.nn as nn
from pytorch_forecasting.models import TemporalFusionTransformer
from torchmetrics import MeanSquaredError
from pytorch_lightning import LightningModule
import torch
import torch.optim as optim
from torchmetrics import MeanSquaredError
from pytorch_lightning import LightningModule
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
import lightning.pytorch as pl # Instead of import pytorch_lightning as pl

from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import LearningRateMonitor
from pytorch_forecasting import DeepAR, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader

# Assuming 'train_data' and 'test_data' are already defined and preprocessed as per your previous code

#



import warnings
import lightning.pytorch as pl
from pytorch_forecasting import TimeSeriesDataSet, DeepAR
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.metrics import SMAPE, MultivariateNormalDistributionLoss
from pytorch_forecasting.data.examples import generate_ar_data

# Suppress warnings and set working directory
warnings.filterwarnings("ignore")
os.chdir("/Users/almazkhalilov/Desktop/uni_sem1_2024/system_optimisation_and_machine_learning/LDI/")

# Generate synthetic data
data = generate_ar_data(seasonality=10.0, timesteps=400, n_series=100, seed=42)
data["static"] = 2
data["date"] = pd.Timestamp("2020-01-01") + pd.to_timedelta(data.time_idx, "D")

# Prepare dataset
training_cutoff = data["time_idx"].max() - 20
training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="value",
    group_ids=["series"],
    max_encoder_length=60,
    max_prediction_length=20,
    time_varying_unknown_reals=["value"],
)

# Prepare dataloaders
train_dataloader = training.to_dataloader(train=True, batch_size=128, num_workers=0)
val_dataloader = training.to_dataloader(train=False, batch_size=128, num_workers=0)

# Initialize model
pl.seed_everything(42)
model = DeepAR.from_dataset(
    training,
    learning_rate=0.01,
    hidden_size=30,
    rnn_layers=2,
    loss=MultivariateNormalDistributionLoss(rank=30),
    log_interval=10,
    log_val_interval=1,
)

# Configure trainer
trainer = pl.Trainer(
    max_epochs=30,
    accelerator="cpu",
    gradient_clip_val=0.1,
    callbacks=[pl.callbacks.EarlyStopping(monitor="val_loss", patience=10)],
)

# Train model
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

Seed set to 42
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                   | Type                               | Params
------------------------------------------------------------------------------
0 | loss                   | MultivariateNormalDistributionLoss | 0     
1 | logging_metrics        | ModuleList                         | 0     
2 | embeddings             | MultiEmbedding                     | 0     
3 | rnn                    | LSTM                               | 11.4 K
4 | distribution_projector | Linear                             | 992   
------------------------------------------------------------------------------
12.4 K    Trainable params
0         Non-trainable params
12.4 K    Total params
0.050     Total estimated model params size (MB)


Epoch 23: 100%|██████████| 235/235 [00:28<00:00,  8.17it/s, v_num=46, train_loss_step=-194., val_loss=-186., train_loss_epoch=-186.]

: 